In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr
torch.manual_seed(42)
generator = torch.Generator()
generator.manual_seed(42)
torch.backends.cudnn.deterministic = True

from functools import partial

from src.trainer import Trainer
from src.Model_class import Model_class
from src.EncDataset import EncoderDataset
from src.Loss_class_encoder import Loss_class_encoder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
df = pd.read_csv('../data/df_to_enc.csv')
df = df.drop(columns = ['Machine failure'])

In [3]:
df.shape

(10000, 60)

In [4]:
HIDDEN_PARAM = 128
LATENT_REPR = 10

BATCH_SIZE = 1024

In [5]:
X_train, X_test = train_test_split(df, shuffle=True, random_state=42)

In [6]:
train_dataset = EncoderDataset(X_train)
val_dataset = EncoderDataset(X_test)

In [7]:
train_dl = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

In [8]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_representation):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, int(hidden_size/2)),
            nn.ReLU(),
            nn.Linear(int(hidden_size/2), int(hidden_size/4)),
            nn.ReLU(),
            nn.Linear(int(hidden_size/4), latent_representation)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_representation, int(hidden_size/4)),
            nn.ReLU(),
            nn.Linear(int(hidden_size/4), int(hidden_size/2)),
            nn.ReLU(),
            nn.Linear(int(hidden_size/2), input_size)
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [9]:
autoencoder = Autoencoder(df.shape[1], HIDDEN_PARAM, LATENT_REPR)

In [10]:
loss = Loss_class_encoder(nn.MSELoss())
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=autoencoder,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.95)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=40)

wandb_init_params = dict(
    name=f'Autoencoder_simple_HidParam-{HIDDEN_PARAM}_Latent-{LATENT_REPR}',
    project="Internship_project",
    dir = '../logs/'
)

In [11]:
trainer = Trainer(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/'
                  )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


In [12]:
trainer.train_model(learning_params)
wandb.finish()

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 91.13it/s]


Epoch: 1 of 40, 0.035 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 94.10it/s]


Epoch: 2 of 40, 0.007 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 94.23it/s]


...

Epoch: 38 of 40, 0.007 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 77.05it/s]


Epoch: 39 of 40, 0.008 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 85.96it/s]


Epoch: 40 of 40, 0.007 min
